In [1]:
from PIL import Image
import time
import numpy as np

s = time.time()

for i in range(1):
    img = Image.open('/media/kyle/store/insta_data/hashtags/#buzzfeast/2019-02-26_23-00-09_UTC.jpg')
    img = img.resize((256,256))

e = time.time()
print(np.asarray(img).transpose(2,0,1).shape)

print(e-s)

FileNotFoundError: [Errno 2] No such file or directory: '/media/kyle/store/insta_data/hashtags/#buzzfeast/2019-02-26_23-00-09_UTC.jpg'

In [ ]:
import os,sys,cv2,gc,json,lzma,re
import numpy as np
import torch
import pickle as pkl
import torch
from imageio import imread
from PIL import Image
import glob

xdim = 256
ydim = 256
metadata = []

dfolder = '/home/jok120/ml/proj/data/old_data/tags/tmp/'

for folder in os.listdir(dfolder):
    n = 0
    n_int = 1

    gc.collect()
    
    path_folder = os.path.join(dfolder,folder)

    if not (os.path.isdir(path_folder)):# and folder.startswith('#')):
        continue
        
    jpgCounter = len(glob.glob1(path_folder,"*.jpg"))
    print(jpgCounter, " JPGs in {}.".format(folder))
    images = torch.empty(jpgCounter,3,xdim,ydim)

    for f in os.listdir(path_folder):
        fname, fext = os.path.splitext(f)

        if fext != '.jpg':
            continue

        #img = imread(os.path.join(dfolder,folder,f)) / 255
        #img = torch.from_numpy(cv2.resize(img, dsize=(ydim,xdim)).transpose(2,0,1)[None,:]).type(torch.FloatTensor)
        img = Image.open(os.path.join(dfolder,folder,f))
        img = img.resize((ydim,xdim))
        img = np.asarray(img) / 255
        img = torch.from_numpy(img.transpose(2,0,1)).type(torch.FloatTensor)

        json_filename = os.path.join(dfolder,folder,re.sub('UTC_(\\d+)','UTC',fname)+'.json.xz')

        try:
            json_str = lzma.open(json_filename).read()
        except:
            continue
        json_data = json.loads(json_str)
        metadata.append(json_data)
        
        images[n] = img
        n += 1

        if n % 100 == 0:
            print('{} - {} images processed'.format(folder,n),end='\r')

    print('{} - finished processing - output shape - {}'.format(folder,images.shape))

    torch.save(images,os.path.join(dfolder,'{}.tch'.format(folder)))
    pkl.dump(metadata,open(os.path.join(dfolder,'{}_metadata.pkl'.format(folder)),'wb'))
    
    images = torch.empty(0,3,xdim,ydim)
    metadata = []

4779  JPGs in lifeandthyme.
lifeandthyme - finished processing - output shape - torch.Size([4779, 3, 256, 256])
388  JPGs in kimberleyhasselbrink.
kimberleyhasselbrink - finished processing - output shape - torch.Size([388, 3, 256, 256])
1500  JPGs in julieskitchen.
julieskitchen - finished processing - output shape - torch.Size([1500, 3, 256, 256])
570  JPGs in hungrybetches.
hungrybetches - finished processing - output shape - torch.Size([570, 3, 256, 256])
1853  JPGs in food_feels.
food_feels - finished processing - output shape - torch.Size([1853, 3, 256, 256])
1  JPGs in local_milk.
local_milk - finished processing - output shape - torch.Size([1, 3, 256, 256])
946  JPGs in izyhossack.
izyhossack - finished processing - output shape - torch.Size([946, 3, 256, 256])
1409  JPGs in candidsbyjo.
candidsbyjo - finished processing - output shape - torch.Size([1409, 3, 256, 256])
304  JPGs in girleatworld.
girleatworld - finished processing - output shape - torch.Size([304, 3, 256, 256])


In [ ]:
import os
import torch
import pickle

dfolder = '/home/jok120/ml/proj/data/old_data/tags/tmp/'
wfolder = '/home/jok120/ml/'

X_train = torch.empty((0,3,64,64))
X_meta = []

N = 1

for f in os.listdir(dfolder):
    fname, fext = os.path.splitext(f)
    
    if fext != '.tch':
        continue
        
    X_train = torch.cat([X_train,torch.load(os.path.join(dfolder,f))])
    X_meta += pickle.load(open(os.path.join(dfolder,fname+'_metadata.pkl'),'rb'))
    print(len(X_train),len(X_meta))
    if len(X_train) > 100000:
        torch.save(X_train,os.path.join(wfolder,'train_data_{}.tch'.format(N)))
        pickle.dump(X_meta,open(os.path.join(wfolder,'train_meta_{}.pkl'.format(N)),'wb'))
        
        N += 1
        
        X_train = torch.empty((0,3,64,64))
        X_meta = []
        
torch.save(X_train,os.path.join(wfolder,'train_data_{}.tch'.format(N)))
pickle.dump(X_meta,open(os.path.join(wfolder,'train_meta_{}.pkl'.format(N)),'wb'))

In [1]:
import torch
a = torch.empty((0,3,64,64))
torch.cat([torch.ones(0,3,64,64),torch.ones(8,3,64,64)]).shape

torch.Size([8, 3, 64, 64])

In [24]:
torch.save(images,os.path.join(dfolder,'data/#homecooking.tch'.format(folder)))
pkl.dump(metadata,open(os.path.join(dfolder,'data/{}_metadata.pkl'.format(folder)),'wb'))